# Final project for Introduction to Quantum Computing (18-819F) Classical Solver (Cplex)

Project Description:
- Scalable and Accurate Generation of Hybrid MPC Protocols with Quantum Integer Programming

### Deps

In [ ]:
#install Cplex
!pip install 'qiskit-optimization[cplex]' --quiet

In [ ]:
import numpy as np
import copy

# Problem modelling imports
from docplex.mp.model import Model
from docplex.mp.linear import LinearExpr
# Qiskit imports
from qiskit import BasicAer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.utils.algorithm_globals import algorithm_globals
from qiskit_optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.problems.variable import VarType
from qiskit_optimization.converters.quadratic_program_to_qubo import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp

### SILPH Reader

In [ ]:
def parse_problem(fname):
    mdl = Model(fname)
    
    # Term Variable Name to Variable 
    # {name: (variable, cost)}
    term_var = {}
    # Conv Variable Name to Variable 
    # {name: (variable, cost)}
    conv_var = {}
    for line in open(fname):
        codes = line.split()
        if codes[0] == "VT":
            # term var
            if not codes[1] in term_var:
                v = mdl.binary_var(codes[1])
                term_var[codes[1]] = (v, float(codes[2]))
        elif codes[0] == "VC":
            # conversion var
            if not codes[1] in conv_var:
                v = mdl.binary_var(codes[1])
                conv_var[codes[1]] = (v, float(codes[2]))
            
        elif codes[0] == "CA":
            # assignment constraint
            if len(codes) == 2:
                # CA term_var
                (v1, _) = term_var[codes[1]]
                mdl.add_constraint(v1 >= 1)
            elif len(codes) == 3:
                # CA term_var term_var
                (v1, _) = term_var[codes[1]]
                (v2, _) = term_var[codes[2]]
                mdl.add_constraint(v1 + v2 >= 1)
            elif len(codes) == 4:
                # CA term_var term_var term_var
                (v1, _) = term_var[codes[1]]
                (v2, _) = term_var[codes[2]]
                (v3, _) = term_var[codes[3]]
                mdl.add_constraint(v1 + v2 + v3 >= 1)
            
        elif codes[0] == "CC":
            # conversion constraint
            (v1, _) = conv_var[codes[1]]
            (v2, _) = term_var[codes[2]]
            (v3, _) = term_var[codes[3]]
            mdl.add_constraint(v1 >= v2 + v3 - 1)
            
    # Create objective funciton
    exp = mdl.linear_expr()
    for (v, cost) in term_var.values():
        exp.add(v*cost)
    for (v, cost) in conv_var.values():
        exp.add(v*cost)
    mdl.minimize(exp)
    return mdl

### QUBO converter

In [ ]:
from qiskit_optimization.converters import InequalityToEquality
from qiskit_optimization.converters import IntegerToBinary
from qiskit_optimization.converters import LinearEqualityToPenalty

In [ ]:
ineq2eq = InequalityToEquality()
int2bin = IntegerToBinary()
lineq2penalty = LinearEqualityToPenalty()
def to_qubo(qp):
    qp_eq = ineq2eq.convert(qp)
    qp_eq_bin = int2bin.convert(qp_eq)
    qubo = lineq2penalty.convert(qp_eq_bin)
    return qubo
    

### Cplex Solver

In [ ]:
import time

def solve(qubo):
    t0 = time.time()
    result = CplexOptimizer().solve(qubo)
    t1 = time.time()
    print("timing:", t1-t0)
    print(result.prettyprint())

In [ ]:
mdl = parse_problem("./toy_1.txt")
# print(mdl.export_as_lp_string())
qp = from_docplex_mp(mdl)
qubo = to_qubo(qp)
solve(qubo)

In [ ]:
mdl = parse_problem("./toy_2.txt")
# print(mdl.export_as_lp_string())
qp = from_docplex_mp(mdl)
qubo = to_qubo(qp)
solve(qubo)

In [ ]:
mdl = parse_problem("./biomatch_4.txt")
# print(mdl.export_as_lp_string())
qp = from_docplex_mp(mdl)
qubo = to_qubo(qp)
solve(qubo)